In [ ]:
import logging
from pyspark.sql.functions import col, when, count, max, avg, datediff, current_date, lit

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

try:
    # Load data from Unity Catalog tables
    logger.info("Loading data from Unity Catalog tables...")
    claims_df = spark.table("genai_demo.jnj.claims")
    demographics_df = spark.table("genai_demo.jnj.demographics")
    policy_df = spark.table("genai_demo.jnj.policy")
    scores_df = spark.table("genai_demo.jnj.scores")
    aiml_insights_df = spark.table("genai_demo.jnj.aiml_insights")

    # Select relevant fields
    logger.info("Selecting relevant fields...")
    claims_df = claims_df.select("Claim_ID", "Policy_ID", "Claim_Date", "Claim_Type", "Claim_Status", "Claim_Amount", "Claim_Payout")
    demographics_df = demographics_df.select("Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City", "State", "Postal_Code", "Date_of_Birth", "Gender", "Marital_Status", "Occupation", "Income_Level", "Customer_Segment")
    policy_df = policy_df.select("Policy_ID", "Customer_ID", "Policy_Type", "Policy_Status", "Policy_Start_Date", "Policy_End_Date", "Policy_Term", "Policy_Premium", "Total_Premium_Paid", "Renewal_Status", "Policy_Addons")
    scores_df = scores_df.select("Customer_ID", "Credit_Score", "Fraud_Score", "Customer_Risk_Score")
    aiml_insights_df = aiml_insights_df.select("Customer_ID", "Churn_Probability", "Next_Best_Offer", "Claims_Fraud_Probability", "Revenue_Potential")

    # Join datasets
    logger.info("Joining datasets...")
    demographics_policy_df = demographics_df.join(policy_df, "Customer_ID", "inner")
    claims_policy_df = claims_df.join(policy_df, "Policy_ID", "inner")

    # Aggregate data
    logger.info("Aggregating data...")
    summary_df = claims_policy_df.groupBy("Customer_ID").agg(
        count("Claim_ID").alias("Total_Claims"),
        count("Policy_ID").alias("Policy_Count"),
        max("Claim_Date").alias("Recent_Claim_Date"),
        avg("Claim_Amount").alias("Average_Claim_Amount")
    )

    # Join summarized data with combined data
    logger.info("Joining summarized data with combined data...")
    combined_df = demographics_policy_df.join(summary_df, "Customer_ID", "inner")

    # Custom calculations
    logger.info("Performing custom calculations...")
    combined_df = combined_df.withColumn("Age", datediff(current_date(), col("Date_of_Birth")) / 365)
    combined_df = combined_df.withColumn("Claim_To_Premium_Ratio", when(col("Total_Premium_Paid") != 0, col("Claim_Amount") / col("Total_Premium_Paid")).otherwise(0))
    combined_df = combined_df.withColumn("Claims_Per_Policy", when(col("Policy_Count") != 0, col("Total_Claims") / col("Policy_Count")).otherwise(0))
    combined_df = combined_df.withColumn("Retention_Rate", lit(0.85))
    combined_df = combined_df.withColumn("Cross_Sell_Opportunities", lit("Multi-Policy Discount, Home Coverage Add-on"))
    combined_df = combined_df.withColumn("Upsell_Potential", lit("Premium Vehicle Coverage"))

    # Integrate AI/ML insights
    logger.info("Integrating AI/ML insights...")
    final_df = combined_df.join(aiml_insights_df, "Customer_ID", "left").join(scores_df, "Customer_ID", "left")

    # Write to Unity Catalog target table
    logger.info("Writing to Unity Catalog target table...")
    spark.sql("DROP TABLE IF EXISTS genai_demo.jnj.customer_360")
    final_df.write.format("delta").mode("overwrite").saveAsTable("genai_demo.jnj.customer_360")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error("An error occurred during the ETL process", exc_info=True)
